In [1]:
import sys
import re
import os

sys.path.insert(0, "src/")
sys.path.insert(1, "external_repos/pytorch_cifar100/")
sys.path.insert(1, "external_repos/pytorch_cifar10/")
import numpy as np
import random
from tqdm.auto import tqdm
from src.data_utils import load_model_checkpoint, load_dict, make_load_path
from src.postprocessing_utils import (
    get_metrics_results,
    uq_funcs_with_names,
    get_uncertainty_scores,
    get_predicted_labels,
    make_aggregation,
    get_raw_scores_dataframe,
    ravel_df,
    create_gt_embeddings,
    get_sampled_combinations_uncertainty_scores,
)
from vectorizer_uncertainty_scores import posterior_predictive

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.metrics import roc_auc_score
from itertools import combinations
from IPython.display import display

pd.set_option("display.max_rows", None)

/home/nkotelevskii/github/uncertainty_from_proper_scoring_rules/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
stty: 'standard input': Inappropriate ioctl for device


In [2]:
training_dataset_names = [
    "cifar10",
    "cifar100",
    "noisy_cifar100",
    "missed_class_cifar10",
    "noisy_cifar10",
]
temperature = 1.0
model_ids = np.arange(20)
list_extraction_datasets = [
    "cifar10",
    "cifar100",
    "svhn",
    "blurred_cifar100",
    "blurred_cifar10",
]
list_ood_datasets = [el for el in list_extraction_datasets]
loss_function_names = ["brier_score", "cross_entropy", "spherical_score"]
use_different_approximations = False
gt_prob_approx = "same"

full_dataframe = None
full_ood_rocauc_dataframe = None
full_mis_rocauc_dataframe = None

In [3]:
ENSEMBLE_COMBINATIONS = [
    (0, 1, 2, 3),
    (4, 5, 6, 7),
    (8, 9, 10, 11),
    (12, 13, 14, 15),
    (16, 17, 18, 19),
]


def get_predicted_labels(
    embeddings_per_dataset: dict,
    training_dataset_name: str,
):
    """
    The function returns predicted labels given embeddings for a given dataset
    """
    pred_labels_dict = {}
    for loss in embeddings_per_dataset:
        pred_labels_dict[loss] = []
        for comb in ENSEMBLE_COMBINATIONS:
            pred_labels = np.argmax(
                posterior_predictive(
                    embeddings_per_dataset[loss][training_dataset_name][list(comb)]
                )[0],
                axis=-1,
            )
            pred_labels_dict[loss].append(pred_labels)
    return pred_labels_dict


def get_missclassification_dataframe(
    ind_dataset: str,
    uq_results: dict,
    true_labels: np.ndarray,
    pred_labels: list[np.ndarray],
) -> pd.DataFrame:
    """
    The function transforms uq_results dict into pd.Dataframe
    with ROC AUC scores of misclassification detection.
    """
    roc_auc_dict = {}

    for uq_name, _ in uq_funcs_with_names:
        roc_auc_dict[uq_name] = {}
        # print(f'Misclassification computed via {uq_name}')

        for loss_ in uq_results[uq_name].keys():
            roc_auc_dict[uq_name][loss_] = []
            for it_ in range(len(uq_results[uq_name][loss_][ind_dataset])):
                y_true = (true_labels != pred_labels[loss_][it_]).astype(np.int32)
                y_score = uq_results[uq_name][loss_][ind_dataset][it_]

                score = roc_auc_score(y_true=y_true, y_score=y_score)
                roc_auc_dict[uq_name][loss_].append(score)

    data_list_misclassification = []
    for metric_name, loss_function in roc_auc_dict.items():
        for loss_function_name, values in loss_function.items():
            data_list_misclassification.append(
                (metric_name, loss_function_name, values)
            )

    # Create a DataFrame
    df_misclassification = pd.DataFrame(
        data_list_misclassification,
        columns=[
            "UQMetric",
            "LossFunction",
            "RocAucScores_array",
        ],
    )
    return df_misclassification

In [4]:
full_dataframe = None

for training_dataset_name in training_dataset_names:
    if training_dataset_name not in [
        "missed_class_cifar10",
        "noisy_cifar10",
        "noisy_cifar100",
    ]:
        architectures = ["resnet18", "vgg"]
        training_dataset_name_aux = training_dataset_name
    else:
        architectures = ["resnet18"]
        training_dataset_name_aux = training_dataset_name.split("_")[-1]
    for architecture in architectures:
        ###
        folder_path = make_load_path(
            architecture=architecture,
            dataset_name=training_dataset_name,
            loss_function_name="NaN",
            model_id="NaN",
        )
        extracted_embeddings_file_path = os.path.join(
            *folder_path.split("/")[:-3],
            "extracted_information_for_notebook_combinations.pkl",
        )

        res_dict = load_dict(extracted_embeddings_file_path)
        uq_results, embeddings_per_dataset, targets_per_dataset = (
            res_dict["uq_results"],
            res_dict["embeddings_per_dataset"],
            res_dict["targets_per_dataset"],
        )
        ###

        max_ind = int(
            targets_per_dataset[training_dataset_name_aux].shape[0] / len(model_ids)
        )
        true_labels = targets_per_dataset[training_dataset_name_aux][:max_ind]

        pred_labels = get_predicted_labels(
            embeddings_per_dataset=embeddings_per_dataset,
            training_dataset_name=training_dataset_name_aux,
        )

        df_misclassification = get_missclassification_dataframe(
            ind_dataset=training_dataset_name_aux,
            uq_results=uq_results,
            true_labels=true_labels,
            pred_labels=pred_labels,
        )
        df_misclassification["architecture"] = architecture
        df_misclassification["training_dataset"] = training_dataset_name

        if full_mis_rocauc_dataframe is None:
            full_mis_rocauc_dataframe = df_misclassification
        else:
            full_mis_rocauc_dataframe = pd.concat(
                [full_mis_rocauc_dataframe, df_misclassification]
            )

In [5]:
full_mis_rocauc_dataframe.shape

(1575, 5)

In [6]:
full_mis_rocauc_dataframe["RocAucScoresMean"] = full_mis_rocauc_dataframe[
    "RocAucScores_array"
].apply(lambda x: np.array(x).mean())
full_mis_rocauc_dataframe["RocAucScoresStd"] = full_mis_rocauc_dataframe[
    "RocAucScores_array"
].apply(lambda x: np.array(x).std())

In [7]:
full_mis_rocauc_dataframe.sample(10)

,UQMetric,LossFunction,RocAucScores_array,architecture,training_dataset,RocAucScoresMean,RocAucScoresStd
42,Bayes Maxprob Inner,brier_score,"[0.8292068551460816, 0.8203732300207724, 0.740...",resnet18,noisy_cifar10,0.797500,0.031445
188,BiasBI Brier,spherical_score,"[0.6697303310739664, 0.7665530918623402, 0.618...",resnet18,noisy_cifar10,0.674174,0.062675
135,Reverse Bregman Information Brier,brier_score,"[0.8423573351457968, 0.8836318662519729, 0.943...",resnet18,missed_class_cifar10,0.910478,0.041025
124,Bregman Information Logscore,cross_entropy,"[0.7724285880541444, 0.7467192091586558, 0.754...",resnet18,noisy_cifar100,0.759900,0.010288
111,Excess Maxprob Inner Inner,brier_score,"[0.5, 0.5, 0.5, 0.5, 0.5]",vgg,cifar10,0.500000,0.000000
190,Bias Maxprob,cross_entropy,"[0.1438103775226446, 0.1455196565955138, 0.142...",vgg,cifar10,0.144981,0.002077
66,Excess Logscore Inner Outer,brier_score,"[0.7269731511109783, 0.7134475324575996, 0.706...",resnet18,noisy_cifar100,0.716347,0.007406
197,MVBI Maxprob,spherical_score,"[0.8069556194667709, 0.8210779266466746, 0.814...",resnet18,cifar10,0.814200,0.010249
48,Bayes Neglog Inner,brier_score,"[0.9271426693388815, 0.9275244647036068, 0.927...",vgg,cifar10,0.928091,0.001107
211,BiasBI Spherical,cross_entropy,"[0.8578697759800266, 0.8641782603994588, 0.861...",vgg,cifar100,0.861691,0.002612


In [8]:
base_score_dict = {
    "cross_entropy": "Logscore",
    "brier_score": "Brier",
    "spherical_score": "Spherical",
}

In [9]:
pattern_baserule = r"(Logscore|Brier|Neglog|Maxprob|Spherical)"
pattern_risk = r"(Total|Bayes|Excess|Reverse Bregman Information|Bregman Information|Expected Pairwise Bregman Information|MVBI|MV|BiasBI|Bias)"

full_mis_rocauc_dataframe["base_rule"] = full_mis_rocauc_dataframe[
    "UQMetric"
].str.extract(pattern_baserule)
full_mis_rocauc_dataframe["RiskType"] = full_mis_rocauc_dataframe[
    "UQMetric"
].str.extract(pattern_risk)
full_mis_rocauc_dataframe["LossFunction"] = full_mis_rocauc_dataframe[
    "LossFunction"
].replace(base_score_dict)

In [10]:
full_mis_rocauc_dataframe.to_csv("./tables/full_mis_rocauc_with_std.csv")

In [27]:
full_mis_rocauc = pd.read_csv("./tables/full_mis_rocauc_with_std.csv", index_col=0)

In [28]:
full_mis_rocauc.training_dataset.unique()

array(['cifar10', 'cifar100', 'noisy_cifar100', 'missed_class_cifar10',
       'noisy_cifar10'], dtype=object)

In [29]:
full_mis_rocauc = full_mis_rocauc[~full_mis_rocauc.UQMetric.str.endswith("Inner Inner")]
# full_mis_rocauc = full_mis_rocauc[full_mis_rocauc.base_rule != 'Neglog']

# full_mis_rocauc.loc[(full_mis_rocauc.RiskType == "Bayes") & full_mis_rocauc.UQMetric.str.endswith("Outer"), "RiskType"] = 'Bayes Outer'
# full_mis_rocauc.loc[(full_mis_rocauc.RiskType == "Bayes") & full_mis_rocauc.UQMetric.str.endswith("Inner"), "RiskType"] = 'Bayes Inner'

# full_mis_rocauc.loc[(full_mis_rocauc.RiskType == "Total") & full_mis_rocauc.UQMetric.str.endswith("Outer"), "RiskType"] = 'Total Outer'
# full_mis_rocauc.loc[(full_mis_rocauc.RiskType == "Total") & full_mis_rocauc.UQMetric.str.endswith("Inner"), "RiskType"] = 'Total Inner'

In [30]:
import sys
import os
import re
import numpy as np

sys.path.insert(0, "src/")

import pandas as pd
from src.table_utils import (
    extract_same_different_dataframes,
    aggregate_over_measures,
)

In [31]:
def collect_scores_into_dict_miss_with_std(
    dataframes_list_,
):
    scores_dict_ = defaultdict(
        list, {val: [] for val in dataframes_list_[0].RiskType.unique()}
    )
    std_dict_ = defaultdict(
        list, {val: [] for val in dataframes_list_[0].RiskType.unique()}
    )

    for dataframe_ in dataframes_list_:
        for ind in dataframe_.training_dataset.unique():
            df_aux_ = dataframe_[(dataframe_["training_dataset"] == ind)]

            mean_rocauc_dict = dict(
                df_aux_.groupby(by=["RiskType"])
                .agg({"RocAucScoresMean": ["mean"]})[("RocAucScoresMean", "mean")]
                .reset_index()
                .values
            )
            std_rocauc_dict = dict(
                df_aux_.groupby(by=["RiskType"])
                .agg({"RocAucScoresStd": ["mean"]})[("RocAucScoresStd", "mean")]
                .reset_index()
                .values
            )
            next_iter = True
            for k in mean_rocauc_dict:
                if k in scores_dict_:
                    scores_dict_[k].append(mean_rocauc_dict[k])
                    std_dict_[k].append(std_rocauc_dict[k])
                    next_iter = False
            if next_iter:
                continue

            scores_dict_["InD"].append(ind)
            scores_dict_["ScoringRule"].append(df_aux_["LossFunction"].unique())

            std_dict_["InD"].append(ind)
            std_dict_["ScoringRule"].append(df_aux_["LossFunction"].unique())
    return scores_dict_, std_dict_

In [32]:
grouped_df = extract_same_different_dataframes(
    dataframe_=full_mis_rocauc,
)

In [33]:
from collections import defaultdict

same_dict_mean, same_dict_std = collect_scores_into_dict_miss_with_std(
    dataframes_list_=[
        grouped_df.logscore_logscore,
        grouped_df.brier_brier,
        grouped_df.spherical_spherical,
    ],
)
same_df_mean = pd.DataFrame.from_dict(same_dict_mean)
same_df_std = pd.DataFrame.from_dict(same_dict_std)

same_agg_df_mean = aggregate_over_measures(
    dataframe_=same_df_mean,
    agg_func_="mean",
    by_=["InD"],
)
same_agg_df_std = aggregate_over_measures(
    dataframe_=same_df_std,
    agg_func_="mean",
    by_=["InD"],
)

different_dict_mean, different_dict_std = collect_scores_into_dict_miss_with_std(
    dataframes_list_=[
        grouped_df.logscore_not_logscore,
        grouped_df.brier_not_brier,
        grouped_df.spherical_not_spherical,
    ],
)
different_df_mean = pd.DataFrame.from_dict(different_dict_mean)
different_df_std = pd.DataFrame.from_dict(different_dict_std)

different_agg_df_mean = aggregate_over_measures(
    dataframe_=different_df_mean,
    agg_func_="mean",
    by_=["InD"],
)
different_agg_df_std = aggregate_over_measures(
    dataframe_=different_df_std,
    agg_func_="mean",
    by_=["InD"],
)

all_dict_mean, all_dict_std = collect_scores_into_dict_miss_with_std(
    dataframes_list_=[
        full_mis_rocauc,
    ],
)
all_df_mean = pd.DataFrame.from_dict(all_dict_mean)
all_df_std = pd.DataFrame.from_dict(all_dict_std)

all_agg_df_mean = aggregate_over_measures(
    dataframe_=all_df_mean,
    agg_func_="mean",
    by_=["InD"],
)
all_agg_df_std = aggregate_over_measures(
    dataframe_=all_df_std,
    agg_func_="mean",
    by_=["InD"],
)

In [34]:
display(all_agg_df_mean)
display(all_agg_df_std)

,Total,Bayes,Excess,Bregman Information,Reverse Bregman Information,Expected Pairwise Bregman Information,Bias,MV,MVBI,BiasBI
,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean
InD,,,,,,,,,,
cifar10,0.935947,0.934619,0.897459,0.897161,0.896719,0.898497,0.660546,0.839788,0.902130,0.761394
cifar100,0.841424,0.854797,0.729444,0.738183,0.720534,0.729614,0.510126,0.676550,0.736675,0.587066
missed_class_cifar10,0.911395,0.899189,0.880219,0.877316,0.881235,0.882104,0.656732,0.824091,0.884527,0.754405
noisy_cifar10,0.777694,0.784407,0.688842,0.690688,0.685141,0.690698,0.511371,0.639119,0.696322,0.595381
noisy_cifar100,0.802213,0.818593,0.638102,0.643024,0.633539,0.637743,0.460808,0.595676,0.649527,0.513263


,Total,Bayes,Excess,Bregman Information,Reverse Bregman Information,Expected Pairwise Bregman Information,Bias,MV,MVBI,BiasBI
,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean
InD,,,,,,,,,,
cifar10,0.002214,0.002226,0.004400,0.004495,0.004349,0.004358,0.004504,0.002690,0.004380,0.003959
cifar100,0.002990,0.002576,0.004303,0.004256,0.004344,0.004310,0.003178,0.004350,0.004996,0.003697
missed_class_cifar10,0.040312,0.060736,0.029749,0.032804,0.027790,0.028653,0.019837,0.027850,0.030412,0.030033
noisy_cifar10,0.035157,0.033661,0.047991,0.048896,0.047608,0.047469,0.044198,0.041879,0.049477,0.048107
noisy_cifar100,0.004808,0.004709,0.007178,0.007099,0.007276,0.007158,0.005123,0.005514,0.007216,0.005997


In [46]:
def enhance_latex_table(input_latex):
    lines = input_latex.split("\n")
    enhanced_lines = []

    for i, line in enumerate(lines):
        if "\\toprule" in line:
            # Add multicolumn headers
            enhanced_lines.append(line)
            enhanced_lines.append(
                r"\multicolumn{2}{c}{Dataset} & \multicolumn{5}{c}{Metrics} \\"
            )
            enhanced_lines.append(r"\cmidrule(lr){1-2} \cmidrule(lr){3-7}")
            continue

        # Add row coloring
        if "\\midrule" in line:
            enhanced_lines.append(line)
            enhanced_lines.append(r"\rowcolor{gray!10}")
        elif "\\bottomrule" in line:
            enhanced_lines.append(r"\end{tabular}")
        else:
            enhanced_lines.append(line)

    return "\n".join(enhanced_lines)


def get_nice_df(df_):
    df_.index = pd.Index(
        data=[
            "CIFAR10",
            "CIFAR100",
            "Missed class CIFAR10",
            "Noisy CIFAR10",
            "Noisy CIFAR100",
        ],
        name="InD",
    )
    # df_.columns = [
    #             # 'Bayes',
    #             # 'Excess',
    #             # 'Total',
    #             'Bayes(O)',
    #             'Bayes(I)',
    #             'Total(O)',
    #             'Total(I)',
    #             'BI',
    #             'RBI',
    #             'EPBI',
    #             # 'Bias',
    #             # 'MV',
    #             # 'MVBI',
    #             # 'BiasBI',
    # ]
    df_ = df_[
        [
            "Bayes",
            "Excess",
            "Total",
            "Bregman Information",
            "Reverse Bregman Information",
            "Expected Pairwise Bregman Information",
        ]
    ]
    df_ = (100 * df_).round(2)

    display(df_)

    return df_, df_.to_latex(float_format="%.2f")

In [47]:
# measures = [
#     'Bayes Outer',
#     'Bayes Inner',
#     'Total Outer',
#     'Total Inner',
#     'Bregman Information',
#     'Reverse Bregman Information',
#     'Expected Pairwise Bregman Information']

measures = [
    "Bayes",
    "Excess",
    "Total",
    "Bregman Information",
    "Reverse Bregman Information",
    "Expected Pairwise Bregman Information",
]

In [50]:
nice_same = get_nice_df(same_agg_df_mean.copy()[measures])
print(enhance_latex_table(nice_same[1]))

,Bayes,Excess,Total,Bregman Information,Reverse Bregman Information,Expected Pairwise Bregman Information
,mean,mean,mean,mean,mean,mean
InD,,,,,,
CIFAR10,93.63,93.22,93.88,93.29,93.14,93.22
CIFAR100,86.32,81.71,86.69,82.45,81.02,81.68
Missed class CIFAR10,90.15,91.09,91.31,91.03,91.09,91.15
Noisy CIFAR10,79.60,73.36,79.24,73.76,73.02,73.30
Noisy CIFAR100,82.81,72.35,82.67,73.10,71.66,72.29


\begin{tabular}{lrrrrrr}
\toprule
\multicolumn{2}{c}{Dataset} & \multicolumn{5}{c}{Metrics} \\
\cmidrule(lr){1-2} \cmidrule(lr){3-7}
 & Bayes & Excess & Total & Bregman Information & Reverse Bregman Information & Expected Pairwise Bregman Information \\
 & mean & mean & mean & mean & mean & mean \\
InD &  &  &  &  &  &  \\
\midrule
\rowcolor{gray!10}
CIFAR10 & 93.63 & 93.22 & 93.88 & 93.29 & 93.14 & 93.22 \\
CIFAR100 & 86.32 & 81.71 & 86.69 & 82.45 & 81.02 & 81.68 \\
Missed class CIFAR10 & 90.15 & 91.09 & 91.31 & 91.03 & 91.09 & 91.15 \\
Noisy CIFAR10 & 79.60 & 73.36 & 79.24 & 73.76 & 73.02 & 73.30 \\
Noisy CIFAR100 & 82.81 & 72.35 & 82.67 & 73.10 & 71.66 & 72.29 \\
\end{tabular}
\end{tabular}



In [51]:
nice_same = get_nice_df(same_agg_df_std[measures].copy())
enhanced_latex = enhance_latex_table(nice_same[1])
print(enhanced_latex)

,Bayes,Excess,Total,Bregman Information,Reverse Bregman Information,Expected Pairwise Bregman Information
,mean,mean,mean,mean,mean,mean
InD,,,,,,
CIFAR10,0.22,0.25,0.21,0.24,0.25,0.25
CIFAR100,0.22,0.32,0.21,0.31,0.32,0.32
Missed class CIFAR10,6.00,3.73,4.17,3.92,3.64,3.63
Noisy CIFAR10,3.42,4.36,3.64,4.39,4.33,4.36
Noisy CIFAR100,0.46,0.73,0.48,0.72,0.74,0.73


\begin{tabular}{lrrrrrr}
\toprule
\multicolumn{2}{c}{Dataset} & \multicolumn{5}{c}{Metrics} \\
\cmidrule(lr){1-2} \cmidrule(lr){3-7}
 & Bayes & Excess & Total & Bregman Information & Reverse Bregman Information & Expected Pairwise Bregman Information \\
 & mean & mean & mean & mean & mean & mean \\
InD &  &  &  &  &  &  \\
\midrule
\rowcolor{gray!10}
CIFAR10 & 0.22 & 0.25 & 0.21 & 0.24 & 0.25 & 0.25 \\
CIFAR100 & 0.22 & 0.32 & 0.21 & 0.31 & 0.32 & 0.32 \\
Missed class CIFAR10 & 6.00 & 3.73 & 4.17 & 3.92 & 3.64 & 3.63 \\
Noisy CIFAR10 & 3.42 & 4.36 & 3.64 & 4.39 & 4.33 & 4.36 \\
Noisy CIFAR100 & 0.46 & 0.73 & 0.48 & 0.72 & 0.74 & 0.73 \\
\end{tabular}
\end{tabular}



In [52]:
nice_same = get_nice_df(different_agg_df_mean[measures].copy())
enhanced_latex = enhance_latex_table(nice_same[1])
print(enhanced_latex)

,Bayes,Excess,Total,Bregman Information,Reverse Bregman Information,Expected Pairwise Bregman Information
,mean,mean,mean,mean,mean,mean
InD,,,,,,
CIFAR10,93.42,88.88,93.52,88.82,88.80,89.01
CIFAR100,85.27,70.75,83.51,71.66,69.81,70.78
Missed class CIFAR10,89.86,87.26,91.10,86.91,87.38,87.48
Noisy CIFAR10,78.15,67.77,77.40,67.90,67.39,68.01
Noisy CIFAR100,81.62,61.68,79.61,62.10,61.28,61.65


\begin{tabular}{lrrrrrr}
\toprule
\multicolumn{2}{c}{Dataset} & \multicolumn{5}{c}{Metrics} \\
\cmidrule(lr){1-2} \cmidrule(lr){3-7}
 & Bayes & Excess & Total & Bregman Information & Reverse Bregman Information & Expected Pairwise Bregman Information \\
 & mean & mean & mean & mean & mean & mean \\
InD &  &  &  &  &  &  \\
\midrule
\rowcolor{gray!10}
CIFAR10 & 93.42 & 88.88 & 93.52 & 88.82 & 88.80 & 89.01 \\
CIFAR100 & 85.27 & 70.75 & 83.51 & 71.66 & 69.81 & 70.78 \\
Missed class CIFAR10 & 89.86 & 87.26 & 91.10 & 86.91 & 87.38 & 87.48 \\
Noisy CIFAR10 & 78.15 & 67.77 & 77.40 & 67.90 & 67.39 & 68.01 \\
Noisy CIFAR100 & 81.62 & 61.68 & 79.61 & 62.10 & 61.28 & 61.65 \\
\end{tabular}
\end{tabular}



In [53]:
nice_same = get_nice_df(different_agg_df_std[measures].copy())
enhanced_latex = enhance_latex_table(nice_same[1])
print(enhanced_latex)

,Bayes,Excess,Total,Bregman Information,Reverse Bregman Information,Expected Pairwise Bregman Information
,mean,mean,mean,mean,mean,mean
InD,,,,,,
CIFAR10,0.22,0.49,0.22,0.50,0.48,0.48
CIFAR100,0.27,0.46,0.32,0.45,0.46,0.46
Missed class CIFAR10,6.09,2.79,4.00,3.12,2.56,2.67
Noisy CIFAR10,3.35,4.91,3.48,5.01,4.87,4.84
Noisy CIFAR100,0.47,0.72,0.48,0.71,0.73,0.71


\begin{tabular}{lrrrrrr}
\toprule
\multicolumn{2}{c}{Dataset} & \multicolumn{5}{c}{Metrics} \\
\cmidrule(lr){1-2} \cmidrule(lr){3-7}
 & Bayes & Excess & Total & Bregman Information & Reverse Bregman Information & Expected Pairwise Bregman Information \\
 & mean & mean & mean & mean & mean & mean \\
InD &  &  &  &  &  &  \\
\midrule
\rowcolor{gray!10}
CIFAR10 & 0.22 & 0.49 & 0.22 & 0.50 & 0.48 & 0.48 \\
CIFAR100 & 0.27 & 0.46 & 0.32 & 0.45 & 0.46 & 0.46 \\
Missed class CIFAR10 & 6.09 & 2.79 & 4.00 & 3.12 & 2.56 & 2.67 \\
Noisy CIFAR10 & 3.35 & 4.91 & 3.48 & 5.01 & 4.87 & 4.84 \\
Noisy CIFAR100 & 0.47 & 0.72 & 0.48 & 0.71 & 0.73 & 0.71 \\
\end{tabular}
\end{tabular}



In [54]:
full_mis_rocauc.base_rule.unique()

array(['Brier', 'Logscore', 'Neglog', 'Maxprob', 'Spherical'],
      dtype=object)